# Análise dos dados das variáveis relacionadas ao IDEB

O notebook contém experimentos com essas variáveis. São utilizadas duas bases de dados:<br>
- Váriáveis derivadas do SAEB (informações socioeconômicas)<br> 
- Variáveis com informações das escolas e professores<br>

In [93]:
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression, mutual_info_regression, f_classif
from sklearn.preprocessing import LabelEncoder
import numpy as np
from numpy import array 

## Importando bases de dados

In [46]:
#Importando variáveis derivadas do SAEB

dados_pessoais = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/variaveis_derivadas_2.0.csv')

dados_pessoais.set_index('ID_ESCOLA', inplace=True)

**Pré-processamento da base de dados pessoais**

In [47]:
#Função que verifica se o ID de uma escola é inválido
#Segundo o Dicionário de Dados da base o SAEB 6 - Os códigos dos Municípios e 
#das Escolas que começam com o dígito "6" são máscaras, isto é, são códigos fictícios.
def retira_id_ficticio(id_escola):
    if id_escola[0] == "6":
        return True
    else:
        return False

In [48]:
#Se retirou as escolas fictícias
for escola in dados_pessoais.index:
    if (retira_id_ficticio(str(escola))):
        dados_pessoais.drop(escola, inplace=True)
        

dados_pessoais

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,...,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
ID_ESCOLA,,,,,,,,,,,,,,,,,,,,,
28012631,2017,2,28,2800100,2,2,1,12,6,15,...,2,3,1,1,0,0,3,3,2,1
28017846,2017,2,28,2800308,1,2,1,21,20,37,...,4,10,2,7,4,1,4,3,3,1
28017854,2017,2,28,2800308,1,2,1,39,62,93,...,7,9,4,10,13,5,15,11,5,10
28018400,2017,2,28,2800308,1,2,1,36,36,71,...,6,10,8,8,4,4,5,3,6,7
28018435,2017,2,28,2800308,1,2,1,56,79,124,...,13,10,14,21,11,10,11,11,7,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2017,2,28,2807105,2,2,1,106,186,253,...,28,37,24,26,21,24,25,18,22,29
28009487,2017,2,28,2807105,2,2,2,18,23,32,...,3,1,4,6,3,7,2,1,4,4
28014839,2017,2,28,2807204,2,2,1,31,42,68,...,8,5,5,11,5,8,6,5,8,5


In [49]:
#Importando variáveis com informações das escolas e professores
dados_escola = pd.read_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/ESCOLAS2017-SE.csv')

dados_escola.set_index('CO_ENTIDADE', inplace=True)

dados_escola

,NU_ANO_CENSO,NO_ENTIDADE,CO_ORGAO_REGIONAL,TP_SITUACAO_FUNCIONAMENTO,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_REGIAO,CO_MESORREGIAO,CO_MICRORREGIAO,CO_UF,...,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,Ideb-2017
CO_ENTIDADE,,,,,,,,,,,,,,,,,,,,,
28017943,2017,COLEGIO AMADEUS LTDA,0.0,1,16/01/2017,24/11/2017,2,2803,28011,28,...,0,0,0,0,45,45,345,45,7,6.9
28010124,2017,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,2.0,1,01/02/2017,08/12/2017,2,2802,28005,28,...,0,0,0,0,24,24,103,24,4,6.6
28018052,2017,COLEGIO SANTANNA DE SERGIPE LTDA,0.0,1,09/01/2017,01/12/2017,2,2803,28011,28,...,0,0,0,0,17,17,89,17,5,6.4
28032497,2017,COLEGIO NOTA DEZ,3.0,1,23/01/2017,15/12/2017,2,2802,28004,28,...,0,0,0,0,11,11,21,11,1,6.1
28017773,2017,COL AMERICANO BATISTA,0.0,1,16/01/2017,01/12/2017,2,2803,28011,28,...,0,0,0,0,22,22,152,22,6,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28022033,2017,COLEGIO ESTADUAL CLEONICE SOARES DA FONSECA,2.0,1,20/02/2017,21/12/2017,2,2803,28012,28,...,0,27,0,0,0,27,326,27,12,4.0
28005759,2017,COLEGIO ESTADUAL EMILIANO GUIMARAES,6.0,1,27/03/2017,16/01/2018,2,2802,28003,28,...,0,14,2,0,0,16,85,16,5,3.0
28018940,2017,COLEGIO ESTADUAL SANTOS DUMONT,0.0,1,02/05/2017,21/03/2018,2,2803,28011,28,...,0,30,0,0,0,30,257,30,8,3.0


#### Pré-processamento da base das escolas

In [50]:
#Retirando valores ausentes
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL                 1
TP_CATEGORIA_ESCOLA_PRIVADA      76
IN_CONVENIADA_PP                 76
TP_CONVENIO_PODER_PUBLICO        96
IN_MANT_ESCOLA_PRIVADA_EMP       76
IN_MANT_ESCOLA_PRIVADA_ONG       76
IN_MANT_ESCOLA_PRIVADA_SIND      76
IN_MANT_ESCOLA_PRIVADA_SIST_S    76
IN_MANT_ESCOLA_PRIVADA_S_FINS    76
CO_ESCOLA_SEDE_VINCULADA         96
CO_IES_OFERTANTE                 96
TP_OCUPACAO_PREDIO_ESCOLAR        1
TP_OCUPACAO_GALPAO               96
IN_PREDIO_COMPARTILHADO           1
IN_FUNDAMENTAL_CICLOS             3
TP_INDIGENA_LINGUA               96
CO_LINGUA_INDIGENA               96
Ideb-2017                         0
dtype: int64

In [51]:
#Retirando colunas que apresentam mais de 70% do valores nulos
dados_escola.dropna(thresh=len(dados_escola)*0.7, axis=1, inplace=True)

#Preenchendo valores ausentes
dados_escola.fillna(-1, inplace=True)

filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.float64]

listOfColumnNames = list(filteredColumns.index)

# retorna o número de valores ausentes para cada coluna
dados_escola[listOfColumnNames].isnull().sum()

CO_ORGAO_REGIONAL             0
TP_OCUPACAO_PREDIO_ESCOLAR    0
IN_PREDIO_COMPARTILHADO       0
IN_FUNDAMENTAL_CICLOS         0
Ideb-2017                     0
dtype: int64

In [52]:
dados_escola['IN_FUNDAMENTAL_CICLOS'].unique()

array([ 0.,  1., -1.])

In [53]:
#Colunas categóricas
filteredColumns = dados_escola.dtypes[dados_escola.dtypes == np.object]

listOfColumnNames = list(filteredColumns.index)

dados_escola[listOfColumnNames]

,NO_ENTIDADE,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO
CO_ENTIDADE,,,
28017943,COLEGIO AMADEUS LTDA,16/01/2017,24/11/2017
28010124,ESCOLA DE EDUCACAO BASICA MONSENHOR JOSE DE SO...,01/02/2017,08/12/2017
28018052,COLEGIO SANTANNA DE SERGIPE LTDA,09/01/2017,01/12/2017
28032497,COLEGIO NOTA DEZ,23/01/2017,15/12/2017
28017773,COL AMERICANO BATISTA,16/01/2017,01/12/2017
...,...,...,...
28022033,COLEGIO ESTADUAL CLEONICE SOARES DA FONSECA,20/02/2017,21/12/2017
28005759,COLEGIO ESTADUAL EMILIANO GUIMARAES,27/03/2017,16/01/2018
28018940,COLEGIO ESTADUAL SANTOS DUMONT,02/05/2017,21/03/2018


In [54]:
#Tratando datas, convertendo para timestamp

inicio = pd.to_datetime(dados_escola['DT_ANO_LETIVO_INICIO'], infer_datetime_format = True)
termino = pd.to_datetime(dados_escola['DT_ANO_LETIVO_TERMINO'], infer_datetime_format = True)

dados_escola['DT_ANO_LETIVO_INICIO'] = inicio.values.astype (np.int64)
dados_escola['DT_ANO_LETIVO_TERMINO'] = termino.values.astype (np.int64)

In [55]:
dados_escola.drop('NO_ENTIDADE', axis=1, inplace=True)

### Unindo as bases de dados 

Foi realizado join nos dois datasets, nesse momento se percebeu que não existem variáveis escolas particulares na base derivada do SAEB

In [56]:
dataset = dados_pessoais.join(dados_escola, how="inner")

In [57]:
dataset

,ID_PROVA_BRASIL,ID_REGIAO,ID_UF,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,...,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf,Ideb-2017
28012631,2017,2,28,2800100,2,2,1,12,6,15,...,0,7,1,0,0,8,22,8,2,3.1
28017846,2017,2,28,2800308,1,2,1,21,20,37,...,0,22,0,0,0,22,121,22,5,3.9
28017854,2017,2,28,2800308,1,2,1,39,62,93,...,0,32,0,0,0,32,301,32,9,2.9
28018400,2017,2,28,2800308,1,2,1,36,36,71,...,0,34,0,0,0,34,383,34,11,3.9
28018435,2017,2,28,2800308,1,2,1,56,79,124,...,0,41,0,0,0,41,476,41,11,3.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2017,2,28,2807105,2,2,1,106,186,253,...,0,46,5,0,0,51,611,51,11,3.5
28009487,2017,2,28,2807105,2,2,2,18,23,32,...,0,12,1,0,0,13,176,13,13,3.9
28014839,2017,2,28,2807204,2,2,1,31,42,68,...,1,16,0,0,1,17,198,17,11,3.5
28010132,2017,2,28,2807402,2,2,1,62,76,110,...,0,27,3,0,0,30,486,30,16,3.5


In [58]:
dataset['ID_DEPENDENCIA_ADM'].unique()

array([2, 1], dtype=int64)

In [60]:
dataset.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa Ideb/variaveis_derivadas_escola_estudante_2.0.csv')

### Discretização dos rótulos, nota do Ideb

**Discretização:** É o processo de transferência de funções contínuas, modelos, variáveis e equações em contrapartes discretas.

In [61]:
# São três classes média, acima da média e abaixo da média
def discretizacao(ideb, mean_ideb):
    classe = []
    for nota in ideb['Ideb-2017']:
        if (mean_ideb == nota):
            classe.append('média')
        elif (mean_ideb < nota):
            classe.append('acima da média')
        else:
            classe.append('abaixo da média')
    return classe

In [62]:
#discretizando as notas do Ideb

Ideb = dataset[['Ideb-2017']]

mean_Ideb = round(Ideb.mean(),1)

classe_ideb = discretizacao(Ideb, mean_Ideb['Ideb-2017'])

Ideb.insert(1, 'classe-Ideb', classe_ideb)

Ideb

,Ideb-2017,classe-Ideb
28012631,3.1,abaixo da média
28017846,3.9,acima da média
28017854,2.9,abaixo da média
28018400,3.9,acima da média
28018435,3.7,acima da média
...,...,...
28009398,3.5,acima da média
28009487,3.9,acima da média
28014839,3.5,acima da média
28010132,3.5,acima da média


In [63]:
dados_escola.columns

Index(['NU_ANO_CENSO', 'CO_ORGAO_REGIONAL', 'TP_SITUACAO_FUNCIONAMENTO',
       'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'CO_REGIAO',
       'CO_MESORREGIAO', 'CO_MICRORREGIAO', 'CO_UF', 'CO_MUNICIPIO',
       ...
       'QtdeDocentesMonit', 'QtdeDocentesEfetivos', 'QtdeDocentesTempo',
       'QtdeDocentesTerceir', 'QtdeDocentesCLT', 'QtdeDocentesTotal',
       'QtdeAlunosTotal', 'QtdeTurmasProf', 'RelAlunProf', 'Ideb-2017'],
      dtype='object', length=171)

## Criação dos modelos

**Serão feitas duas análises:** Apenas com as variáveis socioeconômicas e depois com o dataset completo

Para cada uma serão criados modelos com todos os atributos e também com seleção de atributos, a fim de verificar o de maior acurácia

**-Primeira análise: variáveis socioeconômicas derivadas da prova SAEB**

In [64]:
#Foi retirada colunas do 'ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF'
columns = dataset.columns[3:234]

columns

Index(['ID_MUNICIPIO', 'ID_AREA', 'ID_DEPENDENCIA_ADM', 'ID_LOCALIZACAO',
       'TX_RESP_Q001_A', 'TX_RESP_Q001_B', 'TX_RESP_Q020_A', 'TX_RESP_Q020_B',
       'TX_RESP_Q021_A', 'TX_RESP_Q021_B',
       ...
       'TX_RESP_Q003_B', 'TX_RESP_Q003_C', 'TX_RESP_Q003_D', 'TX_RESP_Q003_E',
       'TX_RESP_Q003_F', 'TX_RESP_Q003_G', 'TX_RESP_Q003_H', 'TX_RESP_Q003_I',
       'TX_RESP_Q003_J', 'TX_RESP_Q003_L'],
      dtype='object', length=231)

In [65]:
X =  dataset[columns]
y = Ideb['classe-Ideb']

X

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,...,TX_RESP_Q003_B,TX_RESP_Q003_C,TX_RESP_Q003_D,TX_RESP_Q003_E,TX_RESP_Q003_F,TX_RESP_Q003_G,TX_RESP_Q003_H,TX_RESP_Q003_I,TX_RESP_Q003_J,TX_RESP_Q003_L
28012631,2800100,2,2,1,12,6,15,4,16,3,...,2,3,1,1,0,0,3,3,2,1
28017846,2800308,1,2,1,21,20,37,4,31,10,...,4,10,2,7,4,1,4,3,3,1
28017854,2800308,1,2,1,39,62,93,10,77,25,...,7,9,4,10,13,5,15,11,5,10
28018400,2800308,1,2,1,36,36,71,2,55,18,...,6,10,8,8,4,4,5,3,6,7
28018435,2800308,1,2,1,56,79,124,14,101,37,...,13,10,14,21,11,10,11,11,7,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2807105,2,2,1,106,186,253,39,198,95,...,28,37,24,26,21,24,25,18,22,29
28009487,2807105,2,2,2,18,23,32,10,31,11,...,3,1,4,6,3,7,2,1,4,4
28014839,2807204,2,2,1,31,42,68,6,57,17,...,8,5,5,11,5,8,6,5,8,5
28010132,2807402,2,2,1,62,76,110,31,92,49,...,5,16,13,17,9,10,13,12,15,6


**Seleção de Atributos:**  É um passo da fase de pré-processamento que tem a finalidade de selecionar o melhor subconjunto de atributos que pode substituir o conjunto de dados original.

In [66]:
#Explicar a escolha do método de seleção de atributos
def selecao_atributos(X, y, quant):
    select = SelectKBest(score_func=f_classif, k=quant)
    z = select.fit_transform(X, y)
    
    print("After selecting best 100 features:", z.shape) 
    
    filter = select.get_support(indices=True)
    features = array(X.columns)
    
    return X.iloc[:, filter]

In [67]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

#### Pré-processamento das variáveis pessoais

In [68]:
def acuracia(resultados, classes):
    #resultados = cross_val_predict(modelo, textos, classes, cv=10)  
    return 'Acurácia do modelo: {} %'.format(metrics.accuracy_score(classes, resultados)*100)

**Normalização:** é mudar os valores das colunas numéricas no conjunto de dados para usar uma escala comum, sem distorcer as diferenças nos intervalos de valores nem perder informações.

**Normalização MinMax:** Para cada recurso, o valor mínimo desse recurso é transformado em 0, o valor máximo é transformado em 1 e todos os outros valores são transformados em um decimal entre 0 e 1.

In [69]:
def minmax(data):
    scaler = MinMaxScaler()
    data_scaler = scaler.fit_transform(data)
    return data_scaler

In [70]:
def inicializar_modelos():
    naive = MultinomialNB()
    random = RandomForestClassifier(n_estimators=150)
    decision_tree = DecisionTreeClassifier()
    svm = SVC(kernel="linear")
    r_logistic = LogisticRegression()
    knn = KNeighborsClassifier(n_neighbors=3)
    return naive, random, decision_tree, svm, r_logistic, knn

**Divisão entre conjunto de treino e teste**

In [71]:
X_scaler = minmax(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [72]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [73]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [74]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 81.25 %
Naive Bayes: Acurácia do modelo: 68.75 %
Decision Tree: Acurácia do modelo: 68.75 %
SVM: Acurácia do modelo: 75.0 %
Logistic Regression: Acurácia do modelo: 81.25 %
KNN: Acurácia do modelo: 50.0 %


**-Primeira Análise: Aplicando Seleção de Atributos**

In [75]:
X_selection = selecao_atributos(X, y, 50)

X_selection

After selecting best 100 features: (76, 50)


C:\Users\maril\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [150] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\maril\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_AREA,ID_LOCALIZACAO,TX_RESP_Q020_B,TX_RESP_Q028_B,TX_RESP_Q043_A,TX_RESP_Q033_A,TX_RESP_Q042_C,TX_RESP_Q049_C,TX_RESP_Q057_C,TX_RESP_Q039_D,...,TX_RESP_Q019_A,TX_RESP_Q019_D,TX_RESP_Q019_F,TX_RESP_Q019_G,TX_RESP_Q023_E,TX_RESP_Q023_F,TX_RESP_Q004_B,TX_RESP_Q004_C,TX_RESP_Q004_G,TX_RESP_Q004_H
28012631,2,1,4,6,4,0,1,0,0,0,...,1,0,4,3,2,2,0,9,0,0
28017846,1,1,4,6,6,7,0,3,3,0,...,1,9,1,4,9,0,1,12,0,0
28017854,1,1,10,9,15,15,5,2,10,1,...,3,17,1,13,24,3,2,22,5,6
28018400,1,1,2,14,3,16,0,1,1,0,...,0,13,8,6,19,7,5,34,0,0
28018435,1,1,14,20,14,8,2,3,3,2,...,2,22,14,13,30,6,6,40,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,1,39,25,15,33,4,2,2,4,...,15,23,15,21,24,5,6,95,15,16
28009487,2,2,10,6,2,6,0,0,0,0,...,4,4,1,4,2,0,0,17,4,1
28014839,2,1,6,3,15,13,1,3,4,0,...,2,14,5,10,10,2,3,24,4,4
28010132,2,1,31,21,27,16,2,2,3,1,...,14,14,7,21,15,2,3,32,10,12


In [76]:
X_scaler = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [77]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [78]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [79]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 87.5 %
Naive Bayes: Acurácia do modelo: 75.0 %
Decision Tree: Acurácia do modelo: 62.5 %
SVM: Acurácia do modelo: 81.25 %
Logistic Regression: Acurácia do modelo: 87.5 %
KNN: Acurácia do modelo: 75.0 %


#### -Segunda análise: Análise com o dataset completo

In [80]:
#Lembrar de tratar datas
X_completo = dataset.drop(['ID_PROVA_BRASIL', 'ID_REGIAO', 'ID_UF', 'DT_ANO_LETIVO_INICIO', 'DT_ANO_LETIVO_TERMINO', 'Ideb-2017'], axis=1)

X_completo

,ID_MUNICIPIO,ID_AREA,ID_DEPENDENCIA_ADM,ID_LOCALIZACAO,TX_RESP_Q001_A,TX_RESP_Q001_B,TX_RESP_Q020_A,TX_RESP_Q020_B,TX_RESP_Q021_A,TX_RESP_Q021_B,...,QtdeDocentesAuxAssist,QtdeDocentesMonit,QtdeDocentesEfetivos,QtdeDocentesTempo,QtdeDocentesTerceir,QtdeDocentesCLT,QtdeDocentesTotal,QtdeAlunosTotal,QtdeTurmasProf,RelAlunProf
28012631,2800100,2,2,1,12,6,15,4,16,3,...,0,0,7,1,0,0,8,22,8,2
28017846,2800308,1,2,1,21,20,37,4,31,10,...,0,0,22,0,0,0,22,121,22,5
28017854,2800308,1,2,1,39,62,93,10,77,25,...,0,0,32,0,0,0,32,301,32,9
28018400,2800308,1,2,1,36,36,71,2,55,18,...,0,0,34,0,0,0,34,383,34,11
28018435,2800308,1,2,1,56,79,124,14,101,37,...,0,0,41,0,0,0,41,476,41,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2807105,2,2,1,106,186,253,39,198,95,...,0,0,46,5,0,0,51,611,51,11
28009487,2807105,2,2,2,18,23,32,10,31,11,...,0,0,12,1,0,0,13,176,13,13
28014839,2807204,2,2,1,31,42,68,6,57,17,...,0,1,16,0,0,1,17,198,17,11
28010132,2807402,2,2,1,62,76,110,31,92,49,...,0,0,27,3,0,0,30,486,30,16


In [81]:
X_transform = minmax(X_completo)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [82]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [83]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

C:\Users\maril\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


KNeighborsClassifier(n_neighbors=3)

In [84]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 68.75 %
Naive Bayes: Acurácia do modelo: 75.0 %
Decision Tree: Acurácia do modelo: 75.0 %
SVM: Acurácia do modelo: 56.25 %
Logistic Regression: Acurácia do modelo: 56.25 %
KNN: Acurácia do modelo: 75.0 %


**-Segunda Análise: Aplicando Seleção de Atributos**

In [85]:
X_selection = selecao_atributos(X_completo, y, 50)

X_selection

After selecting best 100 features: (76, 50)


C:\Users\maril\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [150 231 233 234 237 245 246 247 248 249 250 251 256 259 260 261 262 263
 264 270 271 272 273 282 286 287 289 290 301 305 320 336 339 345 347 348
 350 351 352 353 354 355 358 359 362 364 365 366 367 368 369 370 371 375
 376 377 379 381 382 389 393] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\maril\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,ID_AREA,ID_LOCALIZACAO,TX_RESP_Q020_B,TX_RESP_Q043_A,TX_RESP_Q042_C,TX_RESP_Q039_D,TX_RESP_Q060_D,TX_RESP_Q005_A,TX_RESP_Q005_E,TX_RESP_Q006_E,...,NU_SALAS_EXISTENTES,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_FAX,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_FAX,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_FUNCIONARIOS,IN_COMUM_FUND_AF,QtdeDocentesMest
28012631,2,1,4,4,1,0,0,1,0,0,...,4,1,0,1,0,12,2,17,0,1
28017846,1,1,4,6,0,0,0,0,0,0,...,16,1,0,2,0,5,5,55,1,2
28017854,1,1,10,15,5,1,1,5,1,0,...,20,0,0,0,0,25,3,81,1,2
28018400,1,1,2,3,0,0,1,4,1,0,...,17,1,1,1,1,30,5,75,1,4
28018435,1,1,14,14,2,2,0,2,2,1,...,13,0,0,0,0,34,8,86,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2,1,39,15,4,4,0,10,3,0,...,14,0,0,0,0,33,4,90,0,3
28009487,2,2,10,2,0,0,0,0,0,0,...,6,1,0,2,0,12,4,35,1,1
28014839,2,1,6,15,1,0,0,2,0,0,...,11,1,0,3,0,56,3,55,1,0
28010132,2,1,31,27,2,1,31,6,1,0,...,11,1,0,2,0,8,2,53,1,0


In [86]:
X_transform = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [87]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [88]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [90]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 75.0 %
Naive Bayes: Acurácia do modelo: 81.25 %
Decision Tree: Acurácia do modelo: 68.75 %
SVM: Acurácia do modelo: 81.25 %
Logistic Regression: Acurácia do modelo: 93.75 %
KNN: Acurácia do modelo: 50.0 %


In [91]:
X_selection.columns

Index(['ID_AREA', 'ID_LOCALIZACAO', 'TX_RESP_Q020_B', 'TX_RESP_Q043_A',
       'TX_RESP_Q042_C', 'TX_RESP_Q039_D', 'TX_RESP_Q060_D', 'TX_RESP_Q005_A',
       'TX_RESP_Q005_E', 'TX_RESP_Q006_E', 'TX_RESP_Q009_E', 'TX_RESP_Q010_E',
       'TX_RESP_Q012_B', 'TX_RESP_Q013_B', 'TX_RESP_Q013_C', 'TX_RESP_Q015_A',
       'TX_RESP_Q036_A', 'TX_RESP_Q036_E', 'TX_RESP_Q002_E', 'TX_RESP_Q016_A',
       'TX_RESP_Q019_A', 'TX_RESP_Q019_G', 'TX_RESP_Q023_E', 'TX_RESP_Q023_F',
       'TX_RESP_Q004_B', 'TX_RESP_Q004_G', 'TX_RESP_Q004_H',
       'CO_ORGAO_REGIONAL', 'CO_MESORREGIAO', 'TP_LOCALIZACAO',
       'TP_OCUPACAO_PREDIO_ESCOLAR', 'IN_PREDIO_COMPARTILHADO',
       'IN_ESGOTO_REDE_PUBLICA', 'IN_SALA_DIRETORIA',
       'IN_LABORATORIO_INFORMATICA', 'IN_QUADRA_ESPORTES_COBERTA',
       'IN_QUADRA_ESPORTES', 'IN_BIBLIOTECA', 'IN_BIBLIOTECA_SALA_LEITURA',
       'IN_ALMOXARIFADO', 'NU_SALAS_EXISTENTES', 'IN_EQUIP_IMPRESSORA_MULT',
       'IN_EQUIP_FAX', 'NU_EQUIP_IMPRESSORA_MULT', 'NU_EQUIP_FAX',
   

In [92]:
df_resultado = pd.DataFrame()
df_resultado = y_test.to_frame()
df_resultado['previsto'] = logistic_result

dados_para_exportar = df_resultado.loc[df_resultado['classe-Ideb'] == df_resultado['previsto']].join(dataset[X_selection.columns], how="inner")

In [310]:
dados_para_exportar.to_csv('C:/Users/maril/Documents/ufs/TCC e Pesquisa IDEB/resultado_melhor_analise.csv')

,classe-Ideb,previsto,ID_AREA,ID_LOCALIZACAO,TX_RESP_Q020_B,TX_RESP_Q043_A,TX_RESP_Q042_C,TX_RESP_Q039_D,TX_RESP_Q060_D,TX_RESP_Q005_A,...,NU_SALAS_EXISTENTES,IN_EQUIP_IMPRESSORA_MULT,IN_EQUIP_FAX,NU_EQUIP_IMPRESSORA_MULT,NU_EQUIP_FAX,NU_COMPUTADOR,NU_COMP_ADMINISTRATIVO,NU_FUNCIONARIOS,IN_COMUM_FUND_AF,QtdeDocentesMest
28006720,acima da média,acima da média,2,1,5,3,0,0,0,0,...,10,1,0,3,0,14,3,67,1,1
28013921,abaixo da média,abaixo da média,2,1,13,16,0,0,3,1,...,11,1,1,2,1,16,2,43,1,1
28025903,abaixo da média,abaixo da média,2,1,19,5,5,0,10,5,...,9,1,0,1,0,20,1,41,1,1
28022033,acima da média,acima da média,2,1,11,5,1,1,0,4,...,9,1,1,2,1,42,4,71,1,4
28004809,acima da média,acima da média,2,1,7,4,1,0,0,0,...,9,1,0,1,0,16,4,39,1,0
28001400,abaixo da média,abaixo da média,2,1,13,20,4,1,4,6,...,9,1,1,4,1,30,5,56,1,4
28018516,acima da média,acima da média,1,1,0,4,1,1,0,5,...,14,1,1,2,1,30,8,86,1,3
28000951,acima da média,acima da média,2,1,14,7,0,0,1,3,...,9,1,1,2,1,30,4,37,1,2
28006739,abaixo da média,abaixo da média,2,1,31,17,5,2,75,6,...,11,1,1,3,1,27,7,52,1,1
28013700,abaixo da média,abaixo da média,2,1,22,18,3,1,12,7,...,10,1,0,1,0,31,9,64,1,0


**-Terceira análise: variáveis das escolas e professores**

In [106]:
columns_escolas = dataset.columns[235:404]

X_escolas = dataset[columns_escolas]

X_transform = minmax(X_escolas)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Teste e Treinamento**

In [107]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [108]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [109]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 56.25 %
Naive Bayes: Acurácia do modelo: 50.0 %
Decision Tree: Acurácia do modelo: 25.0 %
SVM: Acurácia do modelo: 37.5 %
Logistic Regression: Acurácia do modelo: 37.5 %
KNN: Acurácia do modelo: 81.25 %


**-Terceira análise: Aplicando seleção de atributos**

In [112]:
X_selection = selecao_atributos(X_escolas , y, 50)

X_selection

After selecting best 100 features: (76, 50)


C:\Users\maril\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:114: UserWarning: Features [  1   4   7  15  16  17  18  19  20  21  26  29  30  31  32  33  34  40
  41  42  43  52  56  57  59  60  71  75  90 106 109 115 117 118 120 121
 122 123 124 125 128 129 132 134 135 136 137 138 139 140 141 145 146 147
 149 151 152 159 163] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx,
C:\Users\maril\anaconda3\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


,CO_ORGAO_REGIONAL,DT_ANO_LETIVO_INICIO,DT_ANO_LETIVO_TERMINO,CO_MESORREGIAO,TP_LOCALIZACAO,TP_REGULAMENTACAO,TP_OCUPACAO_PREDIO_ESCOLAR,IN_LOCAL_FUNC_SALAS_OUTRA_ESC,IN_PREDIO_COMPARTILHADO,IN_ESGOTO_REDE_PUBLICA,...,IN_COMUM_FUND_AF,IN_COMUM_EJA_MEDIO,QtdeDocentesComLicenciatura,QtdeDocentesMest,QtdeDocentesDout,QtdeDocentes,QtdeDocentesEfetivos,QtdeDocentesTotal,QtdeTurmasProf,RelAlunProf
28012631,6.0,1486944000000000000,1513900800000000000,2803,1,1,1.0,0,0.0,1,...,0,1,8,1,0,8,7,8,8,2
28017846,0.0,1494201600000000000,1519776000000000000,2803,1,1,1.0,0,0.0,1,...,1,0,22,2,0,22,22,22,22,5
28017854,0.0,1491782400000000000,1519344000000000000,2803,1,1,1.0,0,0.0,1,...,1,0,28,2,0,32,32,32,32,9
28018400,0.0,1492387200000000000,1519862400000000000,2803,1,1,1.0,0,0.0,1,...,1,0,30,4,0,34,34,34,34,11
28018435,0.0,1489363200000000000,1515196800000000000,2803,1,1,1.0,0,0.0,1,...,1,0,37,5,0,41,41,41,41,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28009398,2.0,1493683200000000000,1518739200000000000,2802,1,1,1.0,0,0.0,1,...,0,0,46,3,0,51,46,51,51,11
28009487,2.0,1490745600000000000,1516838400000000000,2802,2,1,1.0,0,0.0,1,...,1,0,13,1,0,13,12,13,13,13
28014839,5.0,1488412800000000000,1517011200000000000,2803,1,1,1.0,0,0.0,0,...,1,1,16,0,1,16,16,17,17,11
28010132,2.0,1488758400000000000,1516320000000000000,2802,1,1,1.0,0,0.0,1,...,1,0,30,0,0,30,27,30,30,16


In [114]:
X_transform = minmax(X_selection)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, random_state=50, test_size=0.2)

**Fase de Treinamento e Teste**

In [115]:
naive, random, decision_tree, svm, r_logistic, knn = inicializar_modelos()

In [116]:
naive.fit(X_train, y_train)
random.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
svm.fit(X_train, y_train)
r_logistic.fit(X_train, y_train)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [117]:
random_result = random.predict(X_test)
naive_result = naive.predict(X_test)
decision_result = decision_tree.predict(X_test)
svm_result = svm.predict(X_test)
logistic_result = r_logistic.predict(X_test)
knn_result = knn.predict(X_test)

print("Random forest: " + acuracia(random_result, y_test))
print("Naive Bayes: " + acuracia(naive_result, y_test))
print("Decision Tree: " + acuracia(decision_result, y_test))
print("SVM: " + acuracia(svm_result, y_test))
print("Logistic Regression: "+acuracia(logistic_result, y_test))
print("KNN: "+acuracia(knn_result, y_test))

Random forest: Acurácia do modelo: 50.0 %
Naive Bayes: Acurácia do modelo: 56.25 %
Decision Tree: Acurácia do modelo: 56.25 %
SVM: Acurácia do modelo: 50.0 %
Logistic Regression: Acurácia do modelo: 50.0 %
KNN: Acurácia do modelo: 50.0 %
